### **Curating body_inguinal1b.h5ad**

Article:  Multi-scale spatial mapping of cell populations across anatomical sites in healthy human skin and basal cell carcinoma

DOI: https://doi.org/10.1101/2023.08.08.551504

Data Source : https://spatial-skin-atlas.cellgeni.sanger.ac.uk

##### **Mount farm**

mount-farm

##### **Packages required for curation**

In [1]:
#Import all packages required for curation

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess
import math

### **Curation Schema**

##### **X (Matrix Layers)**

##### **AnnData object**

In [3]:
# Load the AnnData object

In [4]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Data/WSSKNKCLsp12887265.h5ad')

In [5]:
# View the AnnData object

In [6]:
adata

AnnData object with n_obs × n_vars = 2247 × 17281
    obs: 'in_tissue', 'array_row', 'array_col', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_spatial'

##### **X- expression matrix**

In [7]:
# View the expression matrix of the anndata object

In [8]:
adata.X

array([[-0.04143578, -0.14912927, -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865],
       [-0.04143578, -0.14912927, -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865],
       [-0.04143578, -0.14912927, -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865],
       ...,
       [-0.04143578,  9.709868  , -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865],
       [-0.04143578, -0.14912927, -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865],
       [-0.04143578, -0.14912927, -0.1523092 , ..., -0.07620841,
        -0.20969167, -0.10276865]], dtype=float32)

In [9]:
#data type of adata.X

In [10]:
type(adata.X)

numpy.ndarray

In [11]:
#convert to csr_matrix

In [12]:
adata.X = csr_matrix(adata.X)

In [13]:
adata.X

<2247x17281 sparse matrix of type '<class 'numpy.float32'>'
	with 38830407 stored elements in Compressed Sparse Row format>

In [14]:
# Print the matrix to check whether they are normalized counts or raw counts. if the matrix has floating numbers,they are normalized counts.if they are integers, they are raw counts.

In [15]:
print(adata.X)

  (0, 0)	-0.04143578
  (0, 1)	-0.14912927
  (0, 2)	-0.1523092
  (0, 3)	-0.054829843
  (0, 4)	-0.08876557
  (0, 5)	-0.0802601
  (0, 6)	-0.09072815
  (0, 7)	4.486433
  (0, 8)	-0.130424
  (0, 9)	-0.2451571
  (0, 10)	-0.062589444
  (0, 11)	-0.53298366
  (0, 12)	-0.3303164
  (0, 13)	-0.27116236
  (0, 14)	-0.039636713
  (0, 15)	-0.2307636
  (0, 16)	-0.17161837
  (0, 17)	-0.04925389
  (0, 18)	-0.25528997
  (0, 19)	-0.22908644
  (0, 20)	-0.6129225
  (0, 21)	-0.22188576
  (0, 22)	-0.10130651
  (0, 23)	-0.33099273
  (0, 24)	-0.18148272
  :	:
  (2246, 17256)	-0.12200865
  (2246, 17257)	-0.16130574
  (2246, 17258)	-0.07863396
  (2246, 17259)	-0.23711382
  (2246, 17260)	-0.048876908
  (2246, 17261)	-0.21771823
  (2246, 17262)	-0.09152858
  (2246, 17263)	-0.1269935
  (2246, 17264)	0.45250377
  (2246, 17265)	0.07137186
  (2246, 17266)	-0.33377802
  (2246, 17267)	-0.2964251
  (2246, 17268)	-0.21868196
  (2246, 17269)	0.16876905
  (2246, 17270)	2.1204429
  (2246, 17271)	0.11724066
  (2246, 17272)	0.223

##### **Raw counts matrix**

In [16]:
# If X has normalized counts, check for the raw counts matrix.

In [17]:
#Here the raw counts are provided in a separate object, load the raw counts matrix

In [18]:
araw= sc.read_10x_h5('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Raw/all/WSSKNKCLsp12887265/filtered_feature_bc_matrix.h5')

/nfs/users/nfs_s/sa32/anaconda/envs/shibla_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [19]:
# view raw object

In [20]:
araw

AnnData object with n_obs × n_vars = 2247 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [21]:
# view raw matrix

In [22]:
araw.X

<2247x36601 sparse matrix of type '<class 'numpy.float32'>'
	with 2996200 stored elements in Compressed Sparse Row format>

In [23]:
print(araw.X)

  (0, 24)	1.0
  (0, 58)	3.0
  (0, 79)	2.0
  (0, 194)	1.0
  (0, 254)	1.0
  (0, 264)	1.0
  (0, 289)	1.0
  (0, 362)	1.0
  (0, 429)	1.0
  (0, 487)	2.0
  (0, 492)	1.0
  (0, 502)	1.0
  (0, 507)	1.0
  (0, 524)	1.0
  (0, 687)	1.0
  (0, 734)	1.0
  (0, 737)	1.0
  (0, 786)	1.0
  (0, 789)	1.0
  (0, 846)	1.0
  (0, 852)	1.0
  (0, 938)	1.0
  (0, 960)	1.0
  (0, 968)	1.0
  (0, 975)	3.0
  :	:
  (2246, 36232)	2.0
  (2246, 36233)	1.0
  (2246, 36298)	2.0
  (2246, 36366)	1.0
  (2246, 36372)	1.0
  (2246, 36378)	6.0
  (2246, 36383)	1.0
  (2246, 36389)	1.0
  (2246, 36398)	6.0
  (2246, 36401)	8.0
  (2246, 36410)	1.0
  (2246, 36411)	1.0
  (2246, 36416)	3.0
  (2246, 36450)	2.0
  (2246, 36559)	7.0
  (2246, 36560)	4.0
  (2246, 36561)	18.0
  (2246, 36562)	16.0
  (2246, 36564)	6.0
  (2246, 36565)	15.0
  (2246, 36566)	9.0
  (2246, 36567)	4.0
  (2246, 36568)	13.0
  (2246, 36569)	2.0
  (2246, 36571)	7.0


In [24]:
# since the raw object is combined one, extract the raw counts for this dataset 

##### **Variables(var)**

In [25]:
#View the var of anndata and raw object

In [26]:
adata.var

""
AL627309.1
LINC01409
LINC01128
LINC00115
FAM41C
...
MAFIP
AC011043.1
AL592183.1
AC007325.4


In [27]:
araw.var

,gene_ids,feature_types,genome
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38
FAM138A,ENSG00000237613,Gene Expression,GRCh38
OR4F5,ENSG00000186092,Gene Expression,GRCh38
AL627309.1,ENSG00000238009,Gene Expression,GRCh38
AL627309.3,ENSG00000239945,Gene Expression,GRCh38
...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38
AC023491.2,ENSG00000278633,Gene Expression,GRCh38
AC007325.1,ENSG00000276017,Gene Expression,GRCh38
AC007325.4,ENSG00000278817,Gene Expression,GRCh38


In [28]:
#Ensembl IDs

In [29]:
gene_info = pd.read_table('/lustre/scratch127/cellgen/cellgeni/shibla/ref_files/2020A.gene_names.tsv')

In [30]:
gene_info

,ensembl_ids,gene,version
0,ENSG00000243485,MIR1302-2HG,2020A
1,ENSG00000237613,FAM138A,2020A
2,ENSG00000186092,OR4F5,2020A
3,ENSG00000238009,AL627309.1,2020A
4,ENSG00000239945,AL627309.3,2020A
...,...,...,...
36596,ENSG00000277836,AC141272.1,2020A
36597,ENSG00000278633,AC023491.2,2020A
36598,ENSG00000276017,AC007325.1,2020A
36599,ENSG00000278817,AC007325.4,2020A


In [31]:
#create a dictionary with gene symbols and ensembl ids from the gene information file

In [32]:
gene_info_genesym_to_ensembl = dict(zip(gene_info['gene'],gene_info['ensembl_ids']))

In [33]:
gene_info_genesym_to_ensembl

{'MIR1302-2HG': 'ENSG00000243485',
 'FAM138A': 'ENSG00000237613',
 'OR4F5': 'ENSG00000186092',
 'AL627309.1': 'ENSG00000238009',
 'AL627309.3': 'ENSG00000239945',
 'AL627309.2': 'ENSG00000239906',
 'AL627309.5': 'ENSG00000241860',
 'AL627309.4': 'ENSG00000241599',
 'AP006222.2': 'ENSG00000286448',
 'AL732372.1': 'ENSG00000236601',
 'OR4F29': 'ENSG00000284733',
 'AC114498.1': 'ENSG00000235146',
 'OR4F16': 'ENSG00000284662',
 'AL669831.2': 'ENSG00000229905',
 'LINC01409': 'ENSG00000237491',
 'FAM87B': 'ENSG00000177757',
 'LINC01128': 'ENSG00000228794',
 'LINC00115': 'ENSG00000225880',
 'FAM41C': 'ENSG00000230368',
 'AL645608.6': 'ENSG00000272438',
 'AL645608.2': 'ENSG00000230699',
 'AL645608.4': 'ENSG00000241180',
 'LINC02593': 'ENSG00000223764',
 'SAMD11': 'ENSG00000187634',
 'NOC2L': 'ENSG00000188976',
 'KLHL17': 'ENSG00000187961',
 'PLEKHN1': 'ENSG00000187583',
 'PERM1': 'ENSG00000187642',
 'AL645608.7': 'ENSG00000272512',
 'HES4': 'ENSG00000188290',
 'ISG15': 'ENSG00000187608',
 'AL6

In [34]:
#Store ensembl ids in a new column in adata.var by matching gene symbols and ensembl ids from the gene information file

In [35]:
adata.var['gene_symbols'] = adata.var_names

In [36]:
araw.var['gene_symbols'] = araw.var_names

In [37]:
araw.var_names = araw.var['gene_ids']

In [38]:
adata.var['ensembl_id'] = adata.var['gene_symbols'].map(gene_info_genesym_to_ensembl)

In [39]:
adata.var

,gene_symbols,ensembl_id
AL627309.1,AL627309.1,ENSG00000238009
LINC01409,LINC01409,ENSG00000237491
LINC01128,LINC01128,ENSG00000228794
LINC00115,LINC00115,ENSG00000225880
FAM41C,FAM41C,ENSG00000230368
...,...,...
MAFIP,MAFIP,ENSG00000274847
AC011043.1,AC011043.1,ENSG00000276256
AL592183.1,AL592183.1,ENSG00000273748
AC007325.4,AC007325.4,ENSG00000278817


In [40]:
adata.var['ensembl_id'].isna().sum()

14

In [41]:
adata.var_names = adata.var['ensembl_id']

In [42]:
adata.var

,gene_symbols,ensembl_id
ensembl_id,,
ENSG00000238009,AL627309.1,ENSG00000238009
ENSG00000237491,LINC01409,ENSG00000237491
ENSG00000228794,LINC01128,ENSG00000228794
ENSG00000225880,LINC00115,ENSG00000225880
ENSG00000230368,FAM41C,ENSG00000230368
...,...,...
ENSG00000274847,MAFIP,ENSG00000274847
ENSG00000276256,AC011043.1,ENSG00000276256
ENSG00000273748,AL592183.1,ENSG00000273748


In [43]:
del adata.var['gene_symbols']
del adata.var['ensembl_id']

In [44]:
del araw.var['gene_ids']
del araw.var['feature_types']
del araw.var['genome']
del araw.var['gene_symbols']

In [45]:
adata.var.index= adata.var.index.drop_duplicates()

In [46]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [47]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


In [48]:
# load the approved genes file

In [49]:
approved_genes = pd.read_csv('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/Endometrium_reference_integrated_atlas/genes_approved.csv')

In [50]:
# Create a dictionary from the approved genes file using the symbols and feature id columns.

In [51]:
genedict = {key: 1 for key in list(approved_genes.feature_id)}

In [52]:
genedict

{'ERCC-00002': 1,
 'ERCC-00003': 1,
 'ERCC-00004': 1,
 'ERCC-00009': 1,
 'ERCC-00012': 1,
 'ERCC-00013': 1,
 'ERCC-00014': 1,
 'ERCC-00016': 1,
 'ERCC-00017': 1,
 'ERCC-00019': 1,
 'ERCC-00022': 1,
 'ERCC-00024': 1,
 'ERCC-00025': 1,
 'ERCC-00028': 1,
 'ERCC-00031': 1,
 'ERCC-00033': 1,
 'ERCC-00034': 1,
 'ERCC-00035': 1,
 'ERCC-00039': 1,
 'ERCC-00040': 1,
 'ERCC-00041': 1,
 'ERCC-00042': 1,
 'ERCC-00043': 1,
 'ERCC-00044': 1,
 'ERCC-00046': 1,
 'ERCC-00048': 1,
 'ERCC-00051': 1,
 'ERCC-00053': 1,
 'ERCC-00054': 1,
 'ERCC-00057': 1,
 'ERCC-00058': 1,
 'ERCC-00059': 1,
 'ERCC-00060': 1,
 'ERCC-00061': 1,
 'ERCC-00062': 1,
 'ERCC-00067': 1,
 'ERCC-00069': 1,
 'ERCC-00071': 1,
 'ERCC-00073': 1,
 'ERCC-00074': 1,
 'ERCC-00075': 1,
 'ERCC-00076': 1,
 'ERCC-00077': 1,
 'ERCC-00078': 1,
 'ERCC-00079': 1,
 'ERCC-00081': 1,
 'ERCC-00083': 1,
 'ERCC-00084': 1,
 'ERCC-00085': 1,
 'ERCC-00086': 1,
 'ERCC-00092': 1,
 'ERCC-00095': 1,
 'ERCC-00096': 1,
 'ERCC-00097': 1,
 'ERCC-00098': 1,
 'ERCC-000

In [53]:
len(genedict)

119799

In [54]:
# Filter out the genes which are not in the approved genes file

In [55]:
var_to_keep_adata = [x for x in adata.var_names if (x in genedict)]
var_to_keep_araw = [x for x in araw.var_names if (x in genedict)]

In [56]:
len(var_to_keep_adata)

17202

In [57]:
len(var_to_keep_araw)

36390

In [58]:
# Modify the anndata object by filtering out the filtered genes. copy the index column values to a new column called gene_symbols

In [59]:
adata = adata[:, var_to_keep_adata].copy()
araw = araw[:, var_to_keep_araw].copy()

In [60]:
#  View the var

In [61]:
adata.var

""
ensembl_id
ENSG00000238009
ENSG00000237491
ENSG00000228794
ENSG00000225880
ENSG00000230368
...
ENSG00000274847
ENSG00000276256
ENSG00000273748


In [62]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


feature is filtered

In [63]:
def add_zero():
	global adata
	global araw
	if araw.shape[1] > adata.shape[1]:
		genes_add = [x for x in araw.var.index.to_list() if x not in adata.var.index.to_list()]
		new_matrix = sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = araw.shape)
		all_genes = adata.var.index.to_list()
		all_genes.extend(genes_add)
		new_var = pd.DataFrame(index=all_genes)
		new_var = pd.merge(new_var, araw.var, left_index=True, right_index=True, how='left')
		new_var['feature_is_filtered'] = False
		new_var.loc[genes_add, 'feature_is_filtered'] = True
		new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=new_var, uns=adata.uns, obsm=adata.obsm)
		if adata.layers:
			for layer in adata.layers:
				new_layer = sparse.csr_matrix((adata.layers[layer].data, adata.layers[layer].indices, adata.layers[layer].indptr), shape = araw.shape)
				new_adata.layers[layer] = new_layer
		new_adata = new_adata[:,araw.var.index.to_list()]
		new_adata.var = new_adata.var.merge(adata.var, left_index=True, right_index=True, how='left')
		adata = new_adata
	else:
		adata.var['feature_is_filtered'] = False

In [64]:
add_zero()

In [65]:
# view var

In [66]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [67]:
list(adata.var['feature_is_filtered'].unique())

[True, False]

In [68]:
True_count = (adata.var['feature_is_filtered']== True).sum()

In [69]:
True_count

19188

In [70]:
araw.var

""
gene_ids
ENSG00000243485
ENSG00000237613
ENSG00000186092
ENSG00000238009
ENSG00000239945
...
ENSG00000277836
ENSG00000278633
ENSG00000276017


#### **Observations(obs) (Cell metadata)**

In [71]:
#view obs

In [72]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_PTGDS+ fibroblasts,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.027607,0.000835,0.016240,0.012226,0.147244,1,2.133156,0.337130,0.037835,0.006145
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.040234,0.007153,0.065781,0.108094,0.150030,3,2.553480,0.195797,0.049437,0.025801
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.184778,0.288332,0.054621,0.020611,0.131938,4,6.481385,0.246743,0.041215,1.573533
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.005892,0.000295,0.025366,0.059051,0.295075,0,2.037784,0.300730,0.171508,0.004244
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.018175,0.011933,0.012768,0.017600,0.370167,1,4.859277,0.711410,0.018917,0.007207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.011243,0.000533,0.029577,0.036673,0.061041,1,1.375480,0.134108,0.080493,0.002528
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.028240,0.000569,0.007587,0.012116,0.049825,6,0.866198,0.055961,0.010036,0.004885
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.024661,0.002372,0.033176,0.022211,0.123283,2,2.424692,0.787070,0.270499,0.008378


In [73]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1'],
      dtype='object')

#### **assay_ontology_term_id**

In [74]:
# add the assay_ontology_term_id column

In [75]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [76]:
# change datatype of the column

In [77]:
adata.obs['assay_ontology_term_id'] = adata.obs['assay_ontology_term_id'].astype('category')

In [78]:
# view adata.obs

In [79]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_MigDC,c2l_Neuronal_SchwannC,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.000835,0.016240,0.012226,0.147244,1,2.133156,0.337130,0.037835,0.006145,EFO:0010961
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.007153,0.065781,0.108094,0.150030,3,2.553480,0.195797,0.049437,0.025801,EFO:0010961
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.288332,0.054621,0.020611,0.131938,4,6.481385,0.246743,0.041215,1.573533,EFO:0010961
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.000295,0.025366,0.059051,0.295075,0,2.037784,0.300730,0.171508,0.004244,EFO:0010961
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.011933,0.012768,0.017600,0.370167,1,4.859277,0.711410,0.018917,0.007207,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.000533,0.029577,0.036673,0.061041,1,1.375480,0.134108,0.080493,0.002528,EFO:0010961
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.000569,0.007587,0.012116,0.049825,6,0.866198,0.055961,0.010036,0.004885,EFO:0010961
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.002372,0.033176,0.022211,0.123283,2,2.424692,0.787070,0.270499,0.008378,EFO:0010961


#### **cell_type_ontology_term_id**

In [80]:
#get the column in adata.obs related. to cell type annotation

In [81]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id'],
      dtype='object')

In [82]:
adata.obsm

AxisArrays with keys: X_spatial

In [83]:
c2l_columns = [col for col in adata.obs.columns if col.startswith('c2l')]

In [84]:
c2l_columns

['c2l_Th',
 'c2l_NK',
 'c2l_APOD+ fibroblasts',
 'c2l_CD8+ T RM',
 'c2l_T reg',
 'c2l_Macro1_2',
 'c2l_DC1',
 'c2l_SFRP2+ fibroblasts',
 'c2l_TAGLN+ pericytes',
 'c2l_POSTN+ fibroblasts',
 'c2l_RGS5+ pericytes',
 'c2l_VEC',
 'c2l_Tc',
 'c2l_ILC_NK',
 'c2l_BC',
 'c2l_Monocytes',
 'c2l_MastC',
 'c2l_Melanocytes',
 'c2l_DC2',
 'c2l_LEC',
 'c2l_PlasmaC',
 'c2l_PTGDS+ fibroblasts',
 'c2l_MigDC',
 'c2l_Neuronal_SchwannC',
 'c2l_SMC',
 'c2l_Skeletal muscle cells',
 'c2l_Suprabasal keratinocytes',
 'c2l_Basal keratinocytes',
 'c2l_Chondrocytes',
 'c2l_IL8+ DC1']

In [85]:
adata.obs['max_c2l_column'] = adata.obs[c2l_columns].idxmax(axis=1)

In [86]:
adata.obs['max_c2l_column_value'] = adata.obs[c2l_columns].max(axis=1)

In [87]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_SMC,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.012226,0.147244,1,2.133156,0.337130,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.108094,0.150030,3,2.553480,0.195797,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.020611,0.131938,4,6.481385,0.246743,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.059051,0.295075,0,2.037784,0.300730,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.017600,0.370167,1,4.859277,0.711410,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.036673,0.061041,1,1.375480,0.134108,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.012116,0.049825,6,0.866198,0.055961,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.022211,0.123283,2,2.424692,0.787070,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365


In [88]:
mapping= {'c2l_Th' :'CL:0000912',
 'c2l_NK':'CL:0000623',
 'c2l_APOD+ fibroblasts':'CL:0000057',
 'c2l_CD8+ T RM':'CL:0001203' ,
 'c2l_T reg':'CL:0000815',
 'c2l_Macro1_2':'CL:0000235',
 'c2l_DC1':'CL:0000990',
 'c2l_SFRP2+ fibroblasts':'CL:0000057',
 'c2l_TAGLN+ pericytes':'CL:0000669',
 'c2l_POSTN+ fibroblasts':'CL:0000057',
 'c2l_RGS5+ pericytes':'CL:0000669',
 'c2l_VEC':'CL:0002139',
 'c2l_Tc':'CL:0000910',
 'c2l_ILC_NK':'CL:0001065', #not NK
 'c2l_BC':'CL:0000646',
 'c2l_Monocytes':'CL:0000576',
 'c2l_MastC':'CL:0000097',
 'c2l_Melanocytes':'CL:1000458',
 'c2l_DC2':'CL:0000784',
 'c2l_LEC':'CL:0002138',
 'c2l_PlasmaC':'CL:0000786',
 'c2l_PTGDS+ fibroblasts':'CL:0000057',
 'c2l_MigDC':'CL:0000451', #not mig
 'c2l_Neuronal_SchwannC':'CL:0002573',
 'c2l_SMC':'CL:0000192',
 'c2l_Skeletal muscle cells':'CL:0000188',
 'c2l_Suprabasal keratinocytes':'CL:4033013',
 'c2l_Basal keratinocytes':'CL:0002187',
 'c2l_Chondrocytes':'CL:0000138',
 'c2l_IL8+ DC1':'CL:0000990'}

In [89]:
# create a dictionary of cell type and ontology term

In [90]:
# add the cell_type_ontology_term_id column

In [91]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['max_c2l_column'].map(mapping)

In [92]:
# change datatype of the column

In [93]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [94]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['CL:0000057',
 'CL:4033013',
 'CL:0002187',
 'CL:0002139',
 'CL:0000192',
 'CL:0000669',
 'CL:0002138',
 'CL:0000623',
 'CL:0002573',
 'CL:0000235',
 'CL:0000138',
 'CL:0000646',
 'CL:0000451']

In [95]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Skeletal muscle cells,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.147244,1,2.133156,0.337130,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.150030,3,2.553480,0.195797,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.131938,4,6.481385,0.246743,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.295075,0,2.037784,0.300730,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.370167,1,4.859277,0.711410,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.061041,1,1.375480,0.134108,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.049825,6,0.866198,0.055961,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.123283,2,2.424692,0.787070,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057


In [96]:
araw.obs

""
AAACAAGTATCTCCCA-1
AAACAGCTTTCAGAAG-1
AAACAGGGTCTATATT-1
AAACCCGAACGAAATC-1
AAACCGGAAATGTTAA-1
...
TTGTTAGCAAATTCGA-1
TTGTTCTAGATACGCT-1
TTGTTGTGTGTCAAGA-1
TTGTTTCACATCCAGG-1


#### **donor_id**

In [97]:
#identify the column in adata.obs which provides donor information

In [98]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id'],
      dtype='object')

In [99]:
# add the donor_id column

In [100]:
adata.obs['donor_id'] = ['body_inguinal_A'] * len(adata.obs)

In [101]:
# change datatype of the column

In [102]:
adata.obs['donor_id'] = adata.obs['donor_id'].astype('category')

In [103]:
# view unique values of donor_id column

In [104]:
list(adata.obs['donor_id'].unique())

['body_inguinal_A']

In [105]:
#view obs

In [106]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,leiden,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,1,2.133156,0.337130,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,3,2.553480,0.195797,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,4,6.481385,0.246743,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0,2.037784,0.300730,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,1,4.859277,0.711410,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,1,1.375480,0.134108,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,6,0.866198,0.055961,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,2,2.424692,0.787070,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A


In [107]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id'],
      dtype='object')

#### **development_stage_ontology_term_id**

In [108]:
# identify the column in adata which corresponds to age

In [109]:
# add the development_stage_ontology_term_id column

In [110]:
adata.obs['development_stage_ontology_term_id'] = ['HsapDv:0000149'] * len(adata.obs)

In [111]:
# change datatype of the column

In [112]:
adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].astype('category')

In [113]:
# view unique values of development_stage_ontology_term_id column

In [114]:
list(adata.obs['development_stage_ontology_term_id'].unique())

['HsapDv:0000149']

In [115]:
# view adata.obs

In [116]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Suprabasal keratinocytes,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,2.133156,0.337130,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,2.553480,0.195797,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,6.481385,0.246743,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,2.037784,0.300730,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,4.859277,0.711410,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,1.375480,0.134108,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.866198,0.055961,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,2.424692,0.787070,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149


#### **disease_ontology_term_id**

In [117]:
# Assign normal since all are healthy patients

In [118]:
# add the disease_ontology_term_id column

In [119]:
adata.obs['disease_ontology_term_id'] = ['PATO:0000461'] * len(adata.obs)

In [120]:
#change data type of column

In [121]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease_ontology_term_id'].astype('category')

In [122]:
# view obs

In [123]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Basal keratinocytes,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.337130,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.195797,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.246743,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.300730,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.711410,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.134108,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.055961,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.787070,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461


#### **is_primary_data**

In [124]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [125]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_Chondrocytes,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.037835,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.049437,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,0.041215,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.171508,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.018917,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.080493,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.010036,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.270499,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True


In [126]:
#change data type of column

In [127]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### **organism_ontology_term_id**

In [128]:
# assign organism id 

In [129]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [130]:
#change data type of column

In [131]:
adata.obs['organism_ontology_term_id'] = adata.obs['organism_ontology_term_id'].astype('category')

In [132]:
# view obs

In [133]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,c2l_IL8+ DC1,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,0.006145,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,0.025801,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,1.573533,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,0.004244,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,0.007207,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,0.002528,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,0.004885,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,0.008378,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606


#### **self_reported_ethnicity_ontology_term_id**

In [134]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [135]:
# change data type

In [136]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [137]:
# view obs

In [138]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,assay_ontology_term_id,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,EFO:0010961,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,EFO:0010961,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown


In [139]:
list(adata.obs['self_reported_ethnicity_ontology_term_id'].unique())

['unknown']

#### **sex_ontology_term_id**

In [140]:
adata.obs['sex_ontology_term_id'] = ['PATO:0000384'] * len(adata.obs)

In [141]:
# change data type

In [142]:
adata.obs['sex_ontology_term_id'] = adata.obs['sex_ontology_term_id'].astype('category')

In [143]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,c2l_SFRP2+ fibroblasts,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,c2l_SFRP2+ fibroblasts,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,c2l_Suprabasal keratinocytes,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,c2l_Suprabasal keratinocytes,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,c2l_Suprabasal keratinocytes,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,c2l_SFRP2+ fibroblasts,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,c2l_SFRP2+ fibroblasts,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,c2l_SFRP2+ fibroblasts,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384


#### **suspension_type**

In [144]:
# since visium suspension type is 'na'

In [145]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

In [146]:
# change data type

In [147]:
adata.obs['suspension_type'] = adata.obs['suspension_type'].astype('category')

In [148]:
# view obs

In [149]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,max_c2l_column_value,cell_type_ontology_term_id,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,3.017945,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,5.044475,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,6.481385,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,2.037784,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,4.859277,CL:4033013,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,2.686604,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,3.027096,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,4.376365,CL:0000057,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na


In [150]:
#### **tissue_type**

In [151]:
adata.obs['tissue_type'] = ['tissue'] * len(adata.obs)

In [152]:
adata.obs['tissue_type'] = adata.obs['tissue_type'].astype('category')

#### **tissue_ontology_term_id**

In [153]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0008337'] * len(adata.obs)

In [154]:
adata.obs['tissue_ontology_term_id'] = adata.obs['tissue_ontology_term_id'].astype('category')

In [155]:
list(adata.obs['tissue_ontology_term_id'].unique())

['UBERON:0008337']

In [156]:
# view obs

In [157]:
adata.obs

,in_tissue,array_row,array_col,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,barcode,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,50,102,658,864,19,2.199074,100,11.574074,AAACAAGTATCTCCCA-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGCTTTCAGAAG-1,1,43,9,971,1454,35,2.407153,199,13.686382,AAACAGCTTTCAGAAG-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGGGTCTATATT-1,1,47,13,4294,13203,448,3.393168,2502,18.950239,AAACAGGGTCTATATT-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCCGAACGAAATC-1,1,45,115,491,608,25,4.111842,79,12.993421,AAACCCGAACGAAATC-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCGGAAATGTTAA-1,1,54,124,915,1330,39,2.932331,202,15.187970,AAACCGGAAATGTTAA-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,22,42,785,1082,32,2.957486,122,11.275415,TTGTTAGCAAATTCGA-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTCTAGATACGCT-1,1,21,3,294,371,18,4.851752,37,9.973046,TTGTTCTAGATACGCT-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTGTGTGTCAAGA-1,1,31,77,691,947,36,3.801478,141,14.889124,TTGTTGTGTGTCAAGA-1,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337


In [158]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb',
       'pct_counts_rb', 'barcode', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts',
       'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1',
       'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'max_c2l_column', 'max_c2l_column_value',
       'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 

In [159]:
del adata.obs['barcode']
del adata.obs['max_c2l_column']
del adata.obs['max_c2l_column_value']
del adata.obs['array_row']
del adata.obs['array_col']

#### **obsm (Embeddings)**

In [160]:
adata.obsm

AxisArrays with keys: X_spatial

In [161]:
adata.obsm.keys()

KeysView(AxisArrays with keys: X_spatial)

#### **uns (Dataset Metadata)**

In [162]:
adata.uns

OverloadedDict, wrapping:
	{'leiden': {'params': {'n_iterations': -1, 'random_state': 0, 'resolution': 1}}, 'log1p': {}, 'neighbors': {'connectivities_key': 'connectivities', 'distances_key': 'distances', 'params': {'method': 'umap', 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}}, 'pca': {'params': {'use_highly_variable': False, 'zero_center': True}, 'variance': array([248.94737  ,  62.53233  ,  34.99212  ,  28.809454 ,  25.810715 ,
        21.010857 ,  18.493204 ,  17.17118  ,  16.714315 ,  16.029642 ,
        15.5579815,  15.244527 ,  15.156795 ,  15.02641  ,  14.986977 ,
        14.843837 ,  14.7929325,  14.768959 ,  14.713739 ,  14.68637  ,
        14.677333 ,  14.645876 ,  14.593494 ,  14.574239 ,  14.534601 ,
        14.524341 ,  14.5168915,  14.469879 ,  14.424895 ,  14.413617 ,
        14.3957615,  14.365015 ,  14.352346 ,  14.3401   ,  14.303295 ,
        14.28521  ,  14.268141 ,  14.242319 ,  14.233406 ,  14.208169 ,
        14.194441 ,  14.180807 ,  14.149607 

In [163]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of the skin from the inguinal part of abdomen stained with H&E'

In [164]:
adata.uns['title'] = 'Visium spatial - body_inguinal_A_b'

In [165]:
adata.uns['default_embedding'] = 'X_spatial'

In [166]:
adata.uns.keys()

dict_keys(['leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap', 'image_caption', 'title', 'default_embedding'])

### **Final checks and adjustments**

In [167]:
adata

AnnData object with n_obs × n_vars = 2247 × 36390
    obs: 'in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK', 'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2', 'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes', 'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc', 'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes', 'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts', 'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC', 'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes', 'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'tissue_t

In [168]:
adata.obs.dtypes

in_tissue                                      int64
n_genes_by_counts                              int32
total_counts                                   int32
total_counts_mt                                int32
pct_counts_mt                                float32
total_counts_rb                                int32
pct_counts_rb                                float32
c2l_Th                                       float64
c2l_NK                                       float64
c2l_APOD+ fibroblasts                        float64
c2l_CD8+ T RM                                float64
c2l_T reg                                    float64
c2l_Macro1_2                                 float64
c2l_DC1                                      float64
c2l_SFRP2+ fibroblasts                       float64
c2l_TAGLN+ pericytes                         float64
c2l_POSTN+ fibroblasts                       float64
c2l_RGS5+ pericytes                          float64
c2l_VEC                                      f

In [169]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

In [170]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed c2l_Th from float64 to float32
changed c2l_NK from float64 to float32
changed c2l_APOD+ fibroblasts from float64 to float32
changed c2l_CD8+ T RM from float64 to float32
changed c2l_T reg from float64 to float32
changed c2l_Macro1_2 from float64 to float32
changed c2l_DC1 from float64 to float32
changed c2l_SFRP2+ fibroblasts from float64 to float32
changed c2l_TAGLN+ pericytes from float64 to float32
changed c2l_POSTN+ fibroblasts from float64 to float32
changed c2l_RGS5+ pericytes from float64 to float32
changed c2l_VEC from float64 to float32
changed c2l_Tc from float64 to float32
changed c2l_ILC_NK from float64 to float32
changed c2l_BC from float64 to float32
changed c2l_Monocytes from float64 to float32
changed c2l_MastC from float64 to float32
changed c2l_Melanocytes from float64 to float32
changed c2l_DC2 from float64 to float32
changed c2l_LEC from float64 to float32
changed c2l_PlasmaC from float64 to float32
changed c2l_PTGDS+ fibroblasts from float64 to float32
chan

In [171]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,658,864,19,2.199074,100,11.574074,0.008785,0.016423,0.120888,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGCTTTCAGAAG-1,1,971,1454,35,2.407153,199,13.686382,0.016532,0.029829,0.342414,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGGGTCTATATT-1,1,4294,13203,448,3.393168,2502,18.950239,0.074150,0.817040,0.590112,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCCGAACGAAATC-1,1,491,608,25,4.111842,79,12.993421,0.004651,0.005268,0.333588,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCGGAAATGTTAA-1,1,915,1330,39,2.932331,202,15.187970,0.011810,0.023496,0.087864,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,785,1082,32,2.957486,122,11.275415,0.005873,0.009868,0.220801,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTCTAGATACGCT-1,1,294,371,18,4.851752,37,9.973046,0.005434,0.011042,0.010824,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTGTGTGTCAAGA-1,1,691,947,36,3.801478,141,14.889124,0.013605,0.023048,0.187702,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337


In [172]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [173]:
adata.var

,feature_is_filtered
ENSG00000243485,True
ENSG00000237613,True
ENSG00000186092,True
ENSG00000238009,False
ENSG00000239945,True
...,...
ENSG00000277836,True
ENSG00000278633,True
ENSG00000276017,True
ENSG00000278817,False


In [174]:
adata.obs

,in_tissue,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,c2l_Th,c2l_NK,c2l_APOD+ fibroblasts,...,donor_id,development_stage_ontology_term_id,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex_ontology_term_id,suspension_type,tissue_type,tissue_ontology_term_id
barcode,,,,,,,,,,,,,,,,,,,,,
AAACAAGTATCTCCCA-1,1,658,864,19,2.199074,100,11.574074,0.008785,0.016423,0.120888,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGCTTTCAGAAG-1,1,971,1454,35,2.407153,199,13.686382,0.016532,0.029829,0.342414,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACAGGGTCTATATT-1,1,4294,13203,448,3.393168,2502,18.950239,0.074150,0.817040,0.590112,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCCGAACGAAATC-1,1,491,608,25,4.111842,79,12.993421,0.004651,0.005268,0.333588,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
AAACCGGAAATGTTAA-1,1,915,1330,39,2.932331,202,15.187970,0.011810,0.023496,0.087864,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTTAGCAAATTCGA-1,1,785,1082,32,2.957486,122,11.275415,0.005873,0.009868,0.220801,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTCTAGATACGCT-1,1,294,371,18,4.851752,37,9.973046,0.005434,0.011042,0.010824,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337
TTGTTGTGTGTCAAGA-1,1,691,947,36,3.801478,141,14.889124,0.013605,0.023048,0.187702,...,body_inguinal_A,HsapDv:0000149,PATO:0000461,True,NCBITaxon:9606,unknown,PATO:0000384,na,tissue,UBERON:0008337


In [175]:
adata.obs.columns

Index(['in_tissue', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'c2l_Th', 'c2l_NK',
       'c2l_APOD+ fibroblasts', 'c2l_CD8+ T RM', 'c2l_T reg', 'c2l_Macro1_2',
       'c2l_DC1', 'c2l_SFRP2+ fibroblasts', 'c2l_TAGLN+ pericytes',
       'c2l_POSTN+ fibroblasts', 'c2l_RGS5+ pericytes', 'c2l_VEC', 'c2l_Tc',
       'c2l_ILC_NK', 'c2l_BC', 'c2l_Monocytes', 'c2l_MastC', 'c2l_Melanocytes',
       'c2l_DC2', 'c2l_LEC', 'c2l_PlasmaC', 'c2l_PTGDS+ fibroblasts',
       'c2l_MigDC', 'c2l_Neuronal_SchwannC', 'c2l_SMC',
       'c2l_Skeletal muscle cells', 'leiden', 'c2l_Suprabasal keratinocytes',
       'c2l_Basal keratinocytes', 'c2l_Chondrocytes', 'c2l_IL8+ DC1',
       'assay_ontology_term_id', 'cell_type_ontology_term_id', 'donor_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology

In [176]:
#check the format of expression matrix

In [177]:
adata.X

<2247x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 38652894 stored elements in Compressed Sparse Row format>

In [178]:
araw.X

<2247x36390 sparse matrix of type '<class 'numpy.float32'>'
	with 2991963 stored elements in Compressed Sparse Row format>

In [179]:
#Copy raw counts to adata.raw

In [180]:
adata.raw = araw

In [181]:
#write the curated object to final_objects folder

In [182]:
del adata.uns['log1p']

In [183]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/spatial-skin/Final_objects/body_inguinal1b.h5ad', compression = 'gzip')